# Utilize target history

Dataset Link here: https://www.kaggle.com/robikscube/ubiquant-parquet

In [ ]:
import pandas as pd
import numpy as np
#import gc

#import matplotlib.pyplot as plt
#import seaborn as sns

#import sklearn
#from sklearn.decomposition import PCA
#import matplotlib.ticker as ticker

from sklearn.model_selection import GroupKFold
#from sklearn import linear_model
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
import lightgbm
from sklearn import linear_model

# EDA

**Data loading**

In [ ]:
train = (pd.read_parquet('../input/ubiquant-parquet/train_low_mem.parquet')
         .sort_values(['time_id', 'investment_id'])
         .drop(columns=['row_id'])
         .query('time_id > 599')
         .reset_index(drop=True));

In [ ]:
train.info()

In [ ]:
train.dtypes

In [ ]:
train.head()

In [ ]:
features = [f'f_{i}' for i in range(300)]
for col in features:
    train[col] = train[col].astype(np.float16)
train.info()

# Utilize target.shift(1)

In [ ]:
train['target_prev']=train.groupby('investment_id')['target'].shift(1)
train = train.dropna(axis=0)

In [ ]:
train

In [ ]:
train.target_prev.hist(bins=20)

In [ ]:
#train = train[train['investment_id'].isin(random_id)].reset_index(drop=True)
train = train.reset_index(drop=True)
groups = train['time_id']

In [ ]:
# target
y = train[['target_prev', 'target']]
X = train.drop(['target_prev', 'target'], axis=1)

In [ ]:
del train

In [ ]:
#from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMRegressor

folds = 5
#models_1 = []
models_2 = []
features = [f'f_{i}' for i in range(300)]
#target = 'target'
#train['preds']=0

# Note we use a group k-fold based on time
kf = GroupKFold(n_splits=folds)
    
for fold, (tr, val) in enumerate(kf.split(X, y, groups)):
    print('FOLD:', fold)
    train_X = X.loc[tr, features]
    #train_y1 = y.loc[tr, 'target_prev']
    train_y2 = y.loc[tr, 'target']
    valid_X = X.loc[val, features]
    #valid_y1 = y.loc[val, 'target_prev']
    valid_y2 = y.loc[val, 'target']
    
    #lgbm = LGBMRegressor(
    #    num_leaves=2 ** np.random.randint(3, 8),
    #    learning_rate = 10 ** (-np.random.uniform(0.1,2)),
    #    n_estimators = 100,
    #    min_child_samples = 1000, 
    #    subsample=np.random.uniform(0.5,1.0), 
    #    subsample_freq=1,
    #    n_jobs= -1
    #)

    #lgbm.fit(train_X, train_y1, eval_set = (valid_X, valid_y1), early_stopping_rounds = 10)
    #model = linear_model.LinearRegression(
    #    n_jobs=-1
    #)
    #model.fit(train_X, train_y1)
    #preds_train = lgbm.predict(train_X)
    #preds_valid = lgbm.predict(valid_X)
    #train.loc[val, "preds"] = preds
    #train_X.loc[:, 'target_prev'] = preds_train
    #valid_X.loc[:, 'target_prev'] = preds_valid
    #print('rmse_1:',np.sqrt(mean_squared_error(valid_y1.values, preds_valid)))
    #print('pearsonr_1:', pearsonr(valid_y1.values, preds_valid)[0])
    #models_1.append(lgbm)
    
    #del preds_train, preds_valid, train_y1, valid_y1, lgbm
    
    lgbm = LGBMRegressor(
        num_leaves=2 ** np.random.randint(3, 8),
        learning_rate = 10 ** (-np.random.uniform(0.1,2)),
        n_estimators = 100,
        min_child_samples = 1000, 
        subsample=np.random.uniform(0.5,1.0), 
        subsample_freq=1,
        n_jobs= -1
    )
    
    lgbm.fit(train_X, train_y2, eval_set = (valid_X, valid_y2), early_stopping_rounds = 10)
    preds = lgbm.predict(valid_X)
    print('rmse_2:',np.sqrt(mean_squared_error(valid_y2.values, preds)))
    print('pearsonr_2:', pearsonr(valid_y2.values, preds)[0])
    lightgbm.plot_importance(lgbm, figsize = (20, 60))
    #if fold == 0:
    #    importance_df= pd.DataFrame(lightgbm.feature_importance(), index=features, columns=fold)
    #else:
    #    importance_df[fold]=lightgbm.feature_importance()
    models_2.append(lgbm)
    del train_X, train_y2, valid_X, valid_y2, lgbm, preds

# Prediction

In [ ]:
import ubiquant
env = ubiquant.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission
for (test_df, sample_prediction_df) in iter_test:
    #test_df.loc[:, 'target_prev']=0
    test_df.loc[:, 'target']=0
    
    #for lgbm_1, lgbm_2 in zip(models_1, models_2):
        #test_df.loc[:, 'target_prev'] = lgbm_1.predict(test_df[features])
        #test_df.loc[:, 'target'] += lgbm_2.predict(test_df[features + ['target_prev']])
        #test_df.loc[:, 'target_prev'] /= len(models_1)
    
    for lgbm_2 in models_2:
        test_df.loc[:, 'target'] += lgbm_2.predict(test_df[features])# + ['target_prev']])
    test_df.loc[:, 'target'] /= len(models_2)
    env.predict(test_df[['row_id','target']])